In [5]:
import numpy as np
import json
import requests
import pandas as pd
from geopy.distance import geodesic

In [6]:
df_resale = pd.read_csv('resale_data.csv')

print(df_resale.shape)
print(df_resale.info())
df_resale.head(2)

(155821, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155821 entries, 0 to 155820
Data columns (total 11 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   month                           155821 non-null  object 
 1   town                            155821 non-null  object 
 2   public_housing_flat_type        155821 non-null  object 
 3   block                           155821 non-null  object 
 4   street_name                     155821 non-null  object 
 5   storey_range                    155821 non-null  object 
 6   floor_area_sqm                  155821 non-null  float64
 7   flat_model                      155821 non-null  object 
 8   lease_commence_date             155821 non-null  int64  
 9   remaining_lease                 155821 non-null  object 
 10  resale_price_Singapore_dollars  155821 non-null  float64
dtypes: float64(2), int64(1), object(8)
memory usage: 13.1+ MB
None


,month,town,public_housing_flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price_Singapore_dollars
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0


In [7]:
def find_postal(lst, filename):
    
    for index, add in enumerate(lst):
        url = 'https://www.onemap.gov.sg/api/common/elastic/search?searchVal=' + add + '&returnGeom=Y&getAddrDetails=Y&pageNum=1'
        print(index, url)
        response = requests.get(url)

        if response.status_code != 200:
            print(f"Error in API response: {response.status_code}")
            continue

        try:
            data = json.loads(response.text)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
            continue

        temp_df = pd.DataFrame.from_dict(data["results"])
        temp_df["address"] = add
    
        if index == 0:
            file = temp_df
        else:
            file = file._append(temp_df)
    file.to_csv(filename + '.csv')



In [8]:
df_resale['Year'] = df_resale['month'].str[:4]
df_resale['mth'] = df_resale['month'].str[5:]
df_resale['address'] = df_resale['block'] + ' ' + df_resale['street_name']
display(df_resale)


,month,town,public_housing_flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price_Singapore_dollars,Year,mth,address
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,2017,01,406 ANG MO KIO AVE 10
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0,2017,01,108 ANG MO KIO AVE 4
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0,2017,01,602 ANG MO KIO AVE 5
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0,2017,01,465 ANG MO KIO AVE 10
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0,2017,01,601 ANG MO KIO AVE 5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155816,2023-06,YISHUN,5 ROOM,511B,YISHUN ST 51,10 TO 12,113.0,Improved,2017,93 years 03 months,645000.0,2023,06,511B YISHUN ST 51
155817,2023-06,YISHUN,5 ROOM,513A,YISHUN ST 51,07 TO 09,120.0,3Gen,2018,93 years 09 months,685000.0,2023,06,513A YISHUN ST 51
155818,2023-06,YISHUN,5 ROOM,613,YISHUN ST 61,01 TO 03,121.0,Improved,1987,62 years 11 months,565000.0,2023,06,613 YISHUN ST 61
155819,2023-06,YISHUN,EXECUTIVE,791,YISHUN AVE 2,04 TO 06,143.0,Apartment,1987,63 years 04 months,845000.0,2023,06,791 YISHUN AVE 2


In [9]:
# Search for postal address of all transacted flats
all_address = list(df_resale['address'])
unique_address = list(set(all_address))
print("Unique addresses:", len(unique_address)) 

find_postal(unique_address, "hdb_address_geoloc")

Unique addresses: 9460
0 https://www.onemap.gov.sg/api/common/elastic/search?searchVal=376A HOUGANG ST 32&returnGeom=Y&getAddrDetails=Y&pageNum=1
1 https://www.onemap.gov.sg/api/common/elastic/search?searchVal=661D JURONG WEST ST 64&returnGeom=Y&getAddrDetails=Y&pageNum=1
2 https://www.onemap.gov.sg/api/common/elastic/search?searchVal=117 BT MERAH CTRL&returnGeom=Y&getAddrDetails=Y&pageNum=1
3 https://www.onemap.gov.sg/api/common/elastic/search?searchVal=221 BISHAN ST 23&returnGeom=Y&getAddrDetails=Y&pageNum=1
4 https://www.onemap.gov.sg/api/common/elastic/search?searchVal=226 BISHAN ST 23&returnGeom=Y&getAddrDetails=Y&pageNum=1
5 https://www.onemap.gov.sg/api/common/elastic/search?searchVal=763 PASIR RIS ST 71&returnGeom=Y&getAddrDetails=Y&pageNum=1
6 https://www.onemap.gov.sg/api/common/elastic/search?searchVal=869 WOODLANDS ST 83&returnGeom=Y&getAddrDetails=Y&pageNum=1
7 https://www.onemap.gov.sg/api/common/elastic/search?searchVal=891B WOODLANDS DR 50&returnGeom=Y&getAddrDetails=Y&

In [10]:
def clean_address(string, dictionary):
    for item in string.split(" "):
        if item in dictionary.keys():
             string = string.replace(item,dictionary[item])
        else:
            string
    return string

In [12]:

hdb = pd.read_csv("hdb_address_geoloc.csv")

# remove repeated items in the geolocation csv file
hdb['block'] = hdb['address'].str.split().str.get(0)
hdb['true_add']= np.where(hdb['BLK_NO'] == hdb['block'], 'yes', 'no')
hdb1 =hdb[hdb['true_add']=='yes'] 
hdbfinal = hdb1.drop_duplicates(subset=['address'], keep='first')
hdbfinal.to_csv('hdb_address_geoloc_final.csv')
text_replace_dict = {"ST." : "ST" }
# Create new resale file with no missing data
df_resale_geopts = pd.read_csv('hdb_address_geoloc_final.csv')
df_resale_geopts = df_resale_geopts[['address', 'LATITUDE', 'LONGITUDE', 'POSTAL']]
df_resale["address_clean"] = [clean_address(val,text_replace_dict) for val in df_resale["address"]]
df_resale['address_clean'] = np.where(df_resale['address_clean'] == "4C ST GEORGE'S LANE", "4C ST. GEORGE'S LANE", 
                                      df_resale['address_clean'])



df_resale_postal= pd.merge(df_resale, df_resale_geopts, how='left', left_on=['address_clean'], right_on=['address'])
df_resale_postal.to_csv('resale-flat_withpostal.csv')

df_resale_postal.head(3)

,month,town,public_housing_flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price_Singapore_dollars,Year,mth,address_x,address_clean,address_y,LATITUDE,LONGITUDE,POSTAL
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,2017,01,406 ANG MO KIO AVE 10,406 ANG MO KIO AVE 10,406 ANG MO KIO AVE 10,1.362005,103.853880,560406.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0,2017,01,108 ANG MO KIO AVE 4,108 ANG MO KIO AVE 4,108 ANG MO KIO AVE 4,1.370966,103.838202,560108.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0,2017,01,602 ANG MO KIO AVE 5,602 ANG MO KIO AVE 5,602 ANG MO KIO AVE 5,1.380709,103.835368,560602.0


In [13]:
display(df_resale_postal)  

,month,town,public_housing_flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price_Singapore_dollars,Year,mth,address_x,address_clean,address_y,LATITUDE,LONGITUDE,POSTAL
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,2017,01,406 ANG MO KIO AVE 10,406 ANG MO KIO AVE 10,406 ANG MO KIO AVE 10,1.362005,103.853880,560406.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0,2017,01,108 ANG MO KIO AVE 4,108 ANG MO KIO AVE 4,108 ANG MO KIO AVE 4,1.370966,103.838202,560108.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0,2017,01,602 ANG MO KIO AVE 5,602 ANG MO KIO AVE 5,602 ANG MO KIO AVE 5,1.380709,103.835368,560602.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0,2017,01,465 ANG MO KIO AVE 10,465 ANG MO KIO AVE 10,465 ANG MO KIO AVE 10,1.366201,103.857201,560465.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0,2017,01,601 ANG MO KIO AVE 5,601 ANG MO KIO AVE 5,601 ANG MO KIO AVE 5,1.381041,103.835132,560601.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155816,2023-06,YISHUN,5 ROOM,511B,YISHUN ST 51,10 TO 12,113.0,Improved,2017,93 years 03 months,645000.0,2023,06,511B YISHUN ST 51,511B YISHUN ST 51,511B YISHUN ST 51,1.415600,103.842414,762511.0
155817,2023-06,YISHUN,5 ROOM,513A,YISHUN ST 51,07 TO 09,120.0,3Gen,2018,93 years 09 months,685000.0,2023,06,513A YISHUN ST 51,513A YISHUN ST 51,513A YISHUN ST 51,1.416040,103.843489,761513.0
155818,2023-06,YISHUN,5 ROOM,613,YISHUN ST 61,01 TO 03,121.0,Improved,1987,62 years 11 months,565000.0,2023,06,613 YISHUN ST 61,613 YISHUN ST 61,613 YISHUN ST 61,1.420067,103.835284,760613.0
155819,2023-06,YISHUN,EXECUTIVE,791,YISHUN AVE 2,04 TO 06,143.0,Apartment,1987,63 years 04 months,845000.0,2023,06,791 YISHUN AVE 2,791 YISHUN AVE 2,791 YISHUN AVE 2,1.420520,103.833564,760791.0


In [14]:
df_hawker = pd.read_csv('list-of-government-markets-hawker-centres.csv')
display(df_hawker)

,name_of_centre,location_of_centre,type_of_centre,owner,no_of_stalls,no_of_cooked_food_stalls,no_of_mkt_produce_stalls
0,Adam Road Food Centre,"2, Adam Road, S(289876)",HC,Government,32,32,0
1,Amoy Street Food Centre,"National Development Building, Annex B, Telok ...",HC,Government,135,134,1
2,Bedok Food Centre,"1, Bedok Road, S(469572)",HC,Government,32,32,0
3,Beo Crescent Market,"38A, Beo Crescent, S(169982)",MHC,Government,94,32,62
4,Berseh Food Centre,"166, Jalan Besar, S(208877)",HC,Government,66,66,0
...,...,...,...,...,...,...,...
102,Hong Lim Market & Food Centre,"Blk 531A, Upper Cross Street, S(051531)",MHC,HDB,143,103,40
103,Kovan Market & Food Centre,"Blk 209, Hougang Street 21, S(530209)",MHC,HDB,186,66,120
104,Pek Kio Market & Food Centre,"Blk 41A, Cambridge Road, S(211041)",MHC,HDB,124,50,74
105,People's Park Food Centre,"Blk 32, New Market Road, S(050032)",HC,HDB,87,87,0


In [15]:
# hdb resale dataset with location
resale_postal = pd.read_csv("resale-flat_withpostal.csv")
resale_postal1 = resale_postal[['address_clean', 'LATITUDE', 'LONGITUDE']]
display(resale_postal1)

,address_clean,LATITUDE,LONGITUDE
0,406 ANG MO KIO AVE 10,1.362005,103.853880
1,108 ANG MO KIO AVE 4,1.370966,103.838202
2,602 ANG MO KIO AVE 5,1.380709,103.835368
3,465 ANG MO KIO AVE 10,1.366201,103.857201
4,601 ANG MO KIO AVE 5,1.381041,103.835132
...,...,...,...
155816,511B YISHUN ST 51,1.415600,103.842414
155817,513A YISHUN ST 51,1.416040,103.843489
155818,613 YISHUN ST 61,1.420067,103.835284
155819,791 YISHUN AVE 2,1.420520,103.833564


In [16]:
from geopy.distance import geodesic
import numpy as np

def find_nearest(house, amenity):
    results = {}

    for index, flat in enumerate(house.iloc[:, 0]):
        flat_loc = (house.iloc[index, 1], house.iloc[index, 2])

        flat_amenity = ['', '', 100]

        for ind, eachloc in enumerate(amenity.iloc[:, 0]):
            amenity_loc = (amenity.iloc[ind, 1], amenity.iloc[ind, 2])

            # Check for NaN values before calculating distance
            if np.isnan(flat_loc[0]) or np.isnan(flat_loc[1]) or np.isnan(amenity_loc[0]) or np.isnan(amenity_loc[1]):
                continue

            distance = geodesic(flat_loc, amenity_loc)

            if distance < flat_amenity[2]:
                flat_amenity[0] = flat
                flat_amenity[1] = eachloc
                flat_amenity[2] = distance

        results[flat] = flat_amenity

    return results


In [17]:
def save_file(data, filename):
    data1 = pd.DataFrame(data)
    data2 = data1.transpose()
    data2.to_csv(filename + '.csv', index=False)

In [24]:
all_hawker_name = list(df_hawker["name_of_centre"])
unique_hawker_name = list(set(all_hawker_name))

print("Unique hawker centre addresses:", len(unique_hawker_name)) 

find_postal(unique_hawker_name, "hdb_hawker_geoloc")

Unique hawker centre addresses: 107
0 https://www.onemap.gov.sg/api/common/elastic/search?searchVal=Blk 90 Whampoa Drive&returnGeom=Y&getAddrDetails=Y&pageNum=1
1 https://www.onemap.gov.sg/api/common/elastic/search?searchVal=Blk 630 Bedok Reservoir Road&returnGeom=Y&getAddrDetails=Y&pageNum=1
2 https://www.onemap.gov.sg/api/common/elastic/search?searchVal=Geylang Serai Market&returnGeom=Y&getAddrDetails=Y&pageNum=1
3 https://www.onemap.gov.sg/api/common/elastic/search?searchVal=Tekka Market&returnGeom=Y&getAddrDetails=Y&pageNum=1
4 https://www.onemap.gov.sg/api/common/elastic/search?searchVal=Blk 58 New Upper Changi Road&returnGeom=Y&getAddrDetails=Y&pageNum=1
5 https://www.onemap.gov.sg/api/common/elastic/search?searchVal=Tanglin Halt Market&returnGeom=Y&getAddrDetails=Y&pageNum=1
6 https://www.onemap.gov.sg/api/common/elastic/search?searchVal=People's Park Food Centre&returnGeom=Y&getAddrDetails=Y&pageNum=1
7 https://www.onemap.gov.sg/api/common/elastic/search?searchVal=Blk 210 Toa P

In [26]:
df_hawker_geoloc = pd.read_csv("hdb_hawker_geoloc.csv")
display(df_hawker_geoloc)

df_hawker_geoloc.rename(columns={'address':'hawkerCentre_name'}, inplace=True)
display(df_hawker_geoloc)

,Unnamed: 0,address,SEARCHVAL,BLK_NO,ROAD_NAME,BUILDING,ADDRESS,POSTAL,X,Y,LATITUDE,LONGITUDE
0,0,Geylang Serai Market,DBS GEYLANG SERAI MARKET AND FOOD CENTRE,1,GEYLANG SERAI,DBS GEYLANG SERAI MARKET AND FOOD CENTRE,1 GEYLANG SERAI DBS GEYLANG SERAI MARKET AND F...,402001,35209.840567,33217.269203,1.316679,103.898103
1,1,Geylang Serai Market,GEYLANG SERAI MARKET,1,GEYLANG SERAI,GEYLANG SERAI MARKET,1 GEYLANG SERAI GEYLANG SERAI MARKET SINGAPORE...,402001,35195.901918,33222.387822,1.316725,103.897977
2,2,Geylang Serai Market,UOB GEYLANG SERAI MARKET,1,GEYLANG SERAI,UOB GEYLANG SERAI MARKET,1 GEYLANG SERAI UOB GEYLANG SERAI MARKET SINGA...,402001,35223.777187,33223.626902,1.316736,103.898228
3,0,Tekka Market,ZHUJIAO CENTRE (TEKKA MARKET),661,BUFFALO ROAD,ZHUJIAO CENTRE (TEKKA MARKET),661 BUFFALO ROAD ZHUJIAO CENTRE (TEKKA MARKET)...,210661,29871.445799,32114.985693,1.306711,103.850134
4,1,Tekka Market,ZHUJIAO CENTRE (TEKKA MARKET),662,BUFFALO ROAD,ZHUJIAO CENTRE (TEKKA MARKET),662 BUFFALO ROAD ZHUJIAO CENTRE (TEKKA MARKET)...,210662,29851.594935,32063.706068,1.306247,103.849956
...,...,...,...,...,...,...,...,...,...,...,...,...
98,0,North Bridge Road Market & Food Centre,NORTH BRIDGE ROAD MARKET,861,NORTH BRIDGE ROAD,NORTH BRIDGE ROAD MARKET,861 NORTH BRIDGE ROAD NORTH BRIDGE ROAD MARKET...,198783,31404.469504,31995.950127,1.305634,103.863909
99,1,North Bridge Road Market & Food Centre,NORTH BRIDGE ROAD MARKET AND FOOD CENTRE,861,NORTH BRIDGE ROAD,NORTH BRIDGE ROAD MARKET AND FOOD CENTRE,861 NORTH BRIDGE ROAD NORTH BRIDGE ROAD MARKET...,198783,31425.607907,32051.953343,1.306141,103.864099
100,0,Chong Pang Market & Food Centre,CHONG PANG MARKET & FOOD CENTRE,104,YISHUN RING ROAD,CHONG PANG MARKET & FOOD CENTRE,104 YISHUN RING ROAD CHONG PANG MARKET & FOOD ...,760104,27467.956893,45907.736997,1.431448,103.828538
101,1,Chong Pang Market & Food Centre,CHONG PANG MARKET & FOOD CENTRE,105,YISHUN RING ROAD,CHONG PANG MARKET & FOOD CENTRE,105 YISHUN RING ROAD CHONG PANG MARKET & FOOD ...,760105,27420.837240,45937.660349,1.431718,103.828114


,Unnamed: 0,hawkerCentre_name,SEARCHVAL,BLK_NO,ROAD_NAME,BUILDING,ADDRESS,POSTAL,X,Y,LATITUDE,LONGITUDE
0,0,Geylang Serai Market,DBS GEYLANG SERAI MARKET AND FOOD CENTRE,1,GEYLANG SERAI,DBS GEYLANG SERAI MARKET AND FOOD CENTRE,1 GEYLANG SERAI DBS GEYLANG SERAI MARKET AND F...,402001,35209.840567,33217.269203,1.316679,103.898103
1,1,Geylang Serai Market,GEYLANG SERAI MARKET,1,GEYLANG SERAI,GEYLANG SERAI MARKET,1 GEYLANG SERAI GEYLANG SERAI MARKET SINGAPORE...,402001,35195.901918,33222.387822,1.316725,103.897977
2,2,Geylang Serai Market,UOB GEYLANG SERAI MARKET,1,GEYLANG SERAI,UOB GEYLANG SERAI MARKET,1 GEYLANG SERAI UOB GEYLANG SERAI MARKET SINGA...,402001,35223.777187,33223.626902,1.316736,103.898228
3,0,Tekka Market,ZHUJIAO CENTRE (TEKKA MARKET),661,BUFFALO ROAD,ZHUJIAO CENTRE (TEKKA MARKET),661 BUFFALO ROAD ZHUJIAO CENTRE (TEKKA MARKET)...,210661,29871.445799,32114.985693,1.306711,103.850134
4,1,Tekka Market,ZHUJIAO CENTRE (TEKKA MARKET),662,BUFFALO ROAD,ZHUJIAO CENTRE (TEKKA MARKET),662 BUFFALO ROAD ZHUJIAO CENTRE (TEKKA MARKET)...,210662,29851.594935,32063.706068,1.306247,103.849956
...,...,...,...,...,...,...,...,...,...,...,...,...
98,0,North Bridge Road Market & Food Centre,NORTH BRIDGE ROAD MARKET,861,NORTH BRIDGE ROAD,NORTH BRIDGE ROAD MARKET,861 NORTH BRIDGE ROAD NORTH BRIDGE ROAD MARKET...,198783,31404.469504,31995.950127,1.305634,103.863909
99,1,North Bridge Road Market & Food Centre,NORTH BRIDGE ROAD MARKET AND FOOD CENTRE,861,NORTH BRIDGE ROAD,NORTH BRIDGE ROAD MARKET AND FOOD CENTRE,861 NORTH BRIDGE ROAD NORTH BRIDGE ROAD MARKET...,198783,31425.607907,32051.953343,1.306141,103.864099
100,0,Chong Pang Market & Food Centre,CHONG PANG MARKET & FOOD CENTRE,104,YISHUN RING ROAD,CHONG PANG MARKET & FOOD CENTRE,104 YISHUN RING ROAD CHONG PANG MARKET & FOOD ...,760104,27467.956893,45907.736997,1.431448,103.828538
101,1,Chong Pang Market & Food Centre,CHONG PANG MARKET & FOOD CENTRE,105,YISHUN RING ROAD,CHONG PANG MARKET & FOOD CENTRE,105 YISHUN RING ROAD CHONG PANG MARKET & FOOD ...,760105,27420.837240,45937.660349,1.431718,103.828114


In [31]:
# For Hawker centres - # find nearest hawker centre to the flat

hawker_postal1 = df_hawker_geoloc[['hawkerCentre_name', 'LATITUDE', 'LONGITUDE']]
display(hawker_postal1)     

map_hawker_postal = find_nearest(resale_postal1 , hawker_postal1)
save_file(map_hawker_postal,"map_hawker_updated")


,hawkerCentre_name,LATITUDE,LONGITUDE
0,Geylang Serai Market,1.316679,103.898103
1,Geylang Serai Market,1.316725,103.897977
2,Geylang Serai Market,1.316736,103.898228
3,Tekka Market,1.306711,103.850134
4,Tekka Market,1.306247,103.849956
...,...,...,...
98,North Bridge Road Market & Food Centre,1.305634,103.863909
99,North Bridge Road Market & Food Centre,1.306141,103.864099
100,Chong Pang Market & Food Centre,1.431448,103.828538
101,Chong Pang Market & Food Centre,1.431718,103.828114


In [32]:
# read hdb property info dataset
# HDB property information contains the location of existing HDB blocks, 
#highest floor level, year of completion, type of building and number of HDB flats (breakdown by flat type) per block etc.

df_property_info = pd.read_csv('hdb-property-information.csv')
df_property_info = df_property_info[['blk_no', 'street', 'max_floor_lvl', 'year_completed', 
                  'market_hawker', 'multistorey_carpark', 'total_dwelling_units']]

In [33]:
# update df_resale
df_resale = pd.read_csv('resale-flat_withpostal.csv')

In [37]:
# Prepare data for merging 
df_hawker = pd.read_csv('map_hawker_updated.csv')
df_hawker.columns = ['address','nearest_hawkerCentre', 'distance_from_hawkerCentre']


In [38]:
# Merge datasets
resale_property_info = pd.merge(df_resale, df_property_info, how='left', 
               left_on=['block', 'street_name'], right_on=['blk_no', 'street'])
resale_and_hawker1 = pd.merge(resale_property_info, df_hawker, how='left', left_on=['address_clean'], right_on=['address'])

# clean after merge
merged_hawker = resale_and_hawker1[:]

merged_hawker['flat_remainlease'] =  merged_hawker['remaining_lease'].str[:2].astype(int)
merged_hawker['est_floor'] =  merged_hawker['storey_range'].str[:2].astype(int)
merged_hawker['distance_hawkerCentre'] =  merged_hawker['distance_from_hawkerCentre'].str[:-2].astype(float)

merged_hawker['hawkerCentre_distribution'] =  np.where(merged_hawker['distance_hawkerCentre'] < 1, 'within-1km', 'More than-1km')

# export 
merged_hawker.to_csv ("final_data_with_hawkerCentre.csv",index = None, header=True)

df_final = pd.read_csv("final_data_with_hawkerCentre.csv")
print(df_final.info())
df_final.head(3)

/var/folders/r6/_v3l5p69253bjvknsl0f4mrr0000gn/T/ipykernel_64540/4059138787.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_hawker['flat_remainlease'] =  merged_hawker['remaining_lease'].str[:2].astype(int)
/var/folders/r6/_v3l5p69253bjvknsl0f4mrr0000gn/T/ipykernel_64540/4059138787.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_hawker['est_floor'] =  merged_hawker['storey_range'].str[:2].astype(int)
/var/folders/r6/_v3l5p69253bjvknsl0f4mrr0000gn/T/ipykernel_64540/4059138787.py:11: Se

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155821 entries, 0 to 155820
Data columns (total 34 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Unnamed: 0                      155821 non-null  int64  
 1   month                           155821 non-null  object 
 2   town                            155821 non-null  object 
 3   public_housing_flat_type        155821 non-null  object 
 4   block                           155821 non-null  object 
 5   street_name                     155821 non-null  object 
 6   storey_range                    155821 non-null  object 
 7   floor_area_sqm                  155821 non-null  float64
 8   flat_model                      155821 non-null  object 
 9   lease_commence_date             155821 non-null  int64  
 10  remaining_lease                 155821 non-null  object 
 11  resale_price_Singapore_dollars  155821 non-null  float64
 12  Year            

,Unnamed: 0,month,town,public_housing_flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,...,market_hawker,multistorey_carpark,total_dwelling_units,address,nearest_hawkerCentre,distance_from_hawkerCentre,flat_remainlease,est_floor,distance_hawkerCentre,hawkerCentre_distribution
0,0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,...,N,N,220,406 ANG MO KIO AVE 10,Blk 409 Ang Mo Kio Ave 10,0.17241064129214373 km,61,10,0.172411,within-1km
1,1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,...,N,N,122,108 ANG MO KIO AVE 4,Blk 724 Ang Mo Kio Ave 6,0.9341909181314921 km,60,1,0.934191,within-1km
2,2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,...,N,N,150,602 ANG MO KIO AVE 5,Sembawang Hill Food Centre,1.166164248332101 km,62,1,1.166164,More than-1km
